In [1]:
import json
import jieba
import joblib
import numpy as np
import lightgbm as lgb
import pandas as pd
import sklearn.metrics as metrics
from sklearn.preprocessing import MultiLabelBinarizer
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from TextCNN import textCNN, train, evaluate
import torch
from embedding import Embedding
from features import (get_basic_feature, get_embedding_feature,
                      get_lda_features, get_tfidf)

In [2]:
stopWords = [
            x.strip() for x in open('./data/stopwords.txt', encoding='utf-8').readlines()
        ]
embedding = Embedding()
embedding.load()
labelToIndex = json.load(
            open('./data/label2id.json', encoding='utf-8'))
ix2label = {v: k for k, v in labelToIndex.items()}
train = pd.read_csv('./data/train.csv',
                                     sep='\t').dropna().reset_index(drop=True).head(300)
dev = pd.read_csv('./data/eval.csv',
                                   sep='\t').dropna().reset_index(drop=True).head(100)
test = pd.read_csv('./data/test.csv',
                                    sep='\t').dropna().reset_index(drop=True).head(100)
exclusive_col = ['text', 'lda', 'bow', 'label']


['———\n', '》），\n', '）÷（１－\n', '”，\n', '）、\n', '＝（\n', ':\n', '→\n', '℃ \n', '&\n', '*\n', '一一\n', '~~~~\n', '’\n', '.\n', '『\n', '.一\n', './\n', '--\n', '』\n', '＝″\n', '【\n', '［＊］\n', '｝＞\n', '［⑤］］\n', '［①Ｄ］\n', 'ｃ］\n', 'ｎｇ昉\n', '＊\n', '//\n', '［\n', '］\n', '［②ｅ］\n', '［②ｇ］\n', '＝｛\n', '}\n', '，也\n', '‘\n', 'Ａ\n', '［①⑥］\n', '［②Ｂ］\n', '［①ａ］\n', '［④ａ］\n', '［①③］\n', '［③ｈ］\n', '③］\n', '１．\n', '－－\n', '［②ｂ］\n', '’‘\n', '×××\n', '［①⑧］\n', '０：２\n', '＝［\n', '［⑤ｂ］\n', '［②ｃ］\n', '［④ｂ］\n', '［②③］\n', '［③ａ］\n', '［④ｃ］\n', '［①⑤］\n', '［①⑦］\n', '［①ｇ］\n', '∈［\n', '［①⑨］\n', '［①④］\n', '［①ｃ］\n', '［②ｆ］\n', '［②⑧］\n', '［②①］\n', '［①Ｃ］\n', '［③ｃ］\n', '［③ｇ］\n', '［②⑤］\n', '［②②］\n', '一.\n', '［①ｈ］\n', '.数\n', '［］\n', '［①Ｂ］\n', '数/\n', '［①ｉ］\n', '［③ｅ］\n', '［①①］\n', '［④ｄ］\n', '［④ｅ］\n', '［③ｂ］\n', '［⑤ａ］\n', '［①Ａ］\n', '［②⑧］\n', '［②⑦］\n', '［①ｄ］\n', '［②ｊ］\n', '〕〔\n', '］［\n', '://\n', '′∈\n', '［②④\n', '［⑤ｅ］\n', '１２％\n', 'ｂ］\n', '...\n', '...................\n', '…………………………………………………③\n', 'ＺＸＦＩＴＬ\n', '［③Ｆ］\n', '」\n', '［①ｏ］\n',

In [3]:
data = get_tfidf(embedding.tfidf, dev)

print(data.head())

E:\Lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', 'sub', 'sup', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｃ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '一一', '一个', '一些', '一切', '一则', '一方面', '一旦', '一来', '一样', '一番', '一直', '一般', '万一', '上下', '不仅', '不但', '不光', '不单', '不只', '不如', '不怕', '不惟', '不成', '不拘', '不比', '不然', '不特', '不独', '不管', '不论', '不过', '不问', '与其', '与否', '与此同时', '两者', '为了', '为什么', '为何', '为着', '乃至', '之一', '之所以', '之类', '乌乎', '也好', '也就是说', '也罢', '于是', '于是乎', '云云', '人家', '什么', '什么样', '从而', '他人', '他们', '以便', '以免', '以及', '以至', '以至于', '以致', '任何', '任凭', '似的', '但是', '何况', '何处', '何时', '作为', '你们', '使得

   label                                               text  tfidf0  tfidf1  \
0   耳鼻喉科  大夫 我 爸 三 个 月 前 做 了 心 胀 支架 手术 || 手术 过程 中 支架 部位 ...     0.0     0.0   
1   神经内科  本人 现 吃药 NUM 年 || 二甲双胍 缓释 片 早晚 各 NUM 片 || 血糖 NU...     0.0     0.0   
2   内分泌科  上 腹 不 定时 疼痛 一个 多 月 || 饭后 加重 || 现 已 去 南京 张 教授 您...     0.0     0.0   
3    皮肤科  感觉 勃起 力度 不够 不 持久 || 在 和 别的 女人 在一起 的 时候 不 做事 想 ...     0.0     0.0   
4  other                       浑身 起 了 小 疙瘩 后背 叫 痒 || 没 吃 过药     0.0     0.0   

   tfidf2  tfidf3  tfidf4  tfidf5    tfidf6  tfidf7  ...  tfidf2781  \
0     0.0     0.0     0.0  0.0000  0.000000     0.0  ...        0.0   
1     0.0     0.0     0.0  0.4103  0.111242     0.0  ...        0.0   
2     0.0     0.0     0.0  0.0000  0.000000     0.0  ...        0.0   
3     0.0     0.0     0.0  0.0000  0.000000     0.0  ...        0.0   
4     0.0     0.0     0.0  0.0000  0.000000     0.0  ...        0.0   

   tfidf2782  tfidf2783  tfidf2784  tfidf2785  tfidf2786  tfidf2787  \
0        0.0        0.0    

In [4]:
data = get_embedding_feature(data, embedding.w2v)
print(data.head())

transform w2v
   label                                               text  tfidf0  tfidf1  \
0   耳鼻喉科  大夫 我 爸 三 个 月 前 做 了 心 胀 支架 手术 || 手术 过程 中 支架 部位 ...     0.0     0.0   
1   神经内科  本人 现 吃药 NUM 年 || 二甲双胍 缓释 片 早晚 各 NUM 片 || 血糖 NU...     0.0     0.0   
2   内分泌科  上 腹 不 定时 疼痛 一个 多 月 || 饭后 加重 || 现 已 去 南京 张 教授 您...     0.0     0.0   
3    皮肤科  感觉 勃起 力度 不够 不 持久 || 在 和 别的 女人 在一起 的 时候 不 做事 想 ...     0.0     0.0   
4  other                       浑身 起 了 小 疙瘩 后背 叫 痒 || 没 吃 过药     0.0     0.0   

   tfidf2  tfidf3  tfidf4  tfidf5    tfidf6  tfidf7  ...  w2v_4_max_290  \
0     0.0     0.0     0.0  0.0000  0.000000     0.0  ...       0.623151   
1     0.0     0.0     0.0  0.4103  0.111242     0.0  ...       0.835323   
2     0.0     0.0     0.0  0.0000  0.000000     0.0  ...       0.554660   
3     0.0     0.0     0.0  0.0000  0.000000     0.0  ...       0.510113   
4     0.0     0.0     0.0  0.0000  0.000000     0.0  ...      -0.557621   

   w2v_4_max_291  w2v_4_max_292  w2v_4_max_293  w2v_4_max_29

In [5]:
data = get_lda_features(data, embedding.lda)
print(data.head())

   label                                               text  tfidf0  tfidf1  \
0   耳鼻喉科  [大夫, 我, 爸, 三, 个, 月, 前, 做, 了, 心, 胀, 支架, 手术, ||,...     0.0     0.0   
1   神经内科  [本人, 现, 吃药, NUM, 年, ||, 二甲双胍, 缓释, 片, 早晚, 各, NU...     0.0     0.0   
2   内分泌科  [上, 腹, 不, 定时, 疼痛, 一个, 多, 月, ||, 饭后, 加重, ||, 现,...     0.0     0.0   
3    皮肤科  [感觉, 勃起, 力度, 不够, 不, 持久, ||, 在, 和, 别的, 女人, 在一起,...     0.0     0.0   
4  other          [浑身, 起, 了, 小, 疙瘩, 后背, 叫, 痒, ||, 没, 吃, 过药]     0.0     0.0   

   tfidf2  tfidf3  tfidf4  tfidf5    tfidf6  tfidf7  ...     lda_1     lda_2  \
0     0.0     0.0     0.0  0.0000  0.000000     0.0  ...  0.212083  0.001596   
1     0.0     0.0     0.0  0.4103  0.111242     0.0  ...  0.192603  0.002526   
2     0.0     0.0     0.0  0.0000  0.000000     0.0  ...  0.001083  0.001083   
3     0.0     0.0     0.0  0.0000  0.000000     0.0  ...  0.001684  0.436954   
4     0.0     0.0     0.0  0.0000  0.000000     0.0  ...  0.006996  0.006996   

      lda_3     lda_4     lda_5     lda_6   